# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 8 2023 9:54AM,256904,15,30013776,"Alliance Pharma, Inc.",Released
1,Feb 8 2023 9:54AM,256904,15,30013775,"Alliance Pharma, Inc.",Released
2,Feb 8 2023 9:54AM,256904,15,30013777,"Alliance Pharma, Inc.",Released
3,Feb 8 2023 9:54AM,256904,15,30013778,"Alliance Pharma, Inc.",Released
4,Feb 8 2023 9:54AM,256904,15,30013779,"Alliance Pharma, Inc.",Released
5,Feb 8 2023 9:54AM,256904,15,30013780,"Alliance Pharma, Inc.",Released
6,Feb 8 2023 9:54AM,256904,15,30013781,"Alliance Pharma, Inc.",Released
7,Feb 8 2023 9:54AM,256904,15,30013782,"Alliance Pharma, Inc.",Released
8,Feb 8 2023 9:54AM,256904,15,30013783,"Alliance Pharma, Inc.",Released
9,Feb 8 2023 9:54AM,256904,15,30013784,"Alliance Pharma, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
41,256900,Released,16
42,256901,Released,2
43,256902,Released,2
44,256903,Released,1
45,256904,Released,34


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
256900,NaN,NaN,16.0
256901,NaN,NaN,2.0
256902,NaN,NaN,2.0
256903,NaN,NaN,1.0
256904,NaN,NaN,34.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256783,25.0,0.0,27.0
256814,0.0,13.0,4.0
256816,1.0,2.0,0.0
256821,0.0,0.0,4.0
256824,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256783,25,0,27
256814,0,13,4
256816,1,2,0
256821,0,0,4
256824,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256783,25,0,27
1,256814,0,13,4
2,256816,1,2,0
3,256821,0,0,4
4,256824,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256783,25,,27
1,256814,,13,4
2,256816,1,2,
3,256821,,,4
4,256824,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 8 2023 9:54AM,256904,15,"Alliance Pharma, Inc."
34,Feb 8 2023 9:52AM,256903,19,"Snap Medical Industries, LLC"
35,Feb 8 2023 9:51AM,256902,10,"Snap Medical Industries, LLC"
37,Feb 8 2023 9:49AM,256901,19,"Methapharm, Inc."
39,Feb 8 2023 9:45AM,256827,15,"Virtus Pharmaceuticals, LLC"
55,Feb 8 2023 9:33AM,256900,15,"Brookfield Pharmaceuticals, LLC"
71,Feb 8 2023 9:28AM,256899,10,"Citieffe, Inc."
72,Feb 8 2023 9:25AM,256898,22,"NBTY Global, Inc."
73,Feb 8 2023 9:15AM,256897,10,"Methapharm, Inc."
90,Feb 8 2023 9:09AM,256892,18,Innova Softgel LLC


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 8 2023 9:54AM,256904,15,"Alliance Pharma, Inc.",,,34
1,Feb 8 2023 9:52AM,256903,19,"Snap Medical Industries, LLC",,,1
2,Feb 8 2023 9:51AM,256902,10,"Snap Medical Industries, LLC",,,2
3,Feb 8 2023 9:49AM,256901,19,"Methapharm, Inc.",,,2
4,Feb 8 2023 9:45AM,256827,15,"Virtus Pharmaceuticals, LLC",,,16
5,Feb 8 2023 9:33AM,256900,15,"Brookfield Pharmaceuticals, LLC",,,16
6,Feb 8 2023 9:28AM,256899,10,"Citieffe, Inc.",,,1
7,Feb 8 2023 9:25AM,256898,22,"NBTY Global, Inc.",,,1
8,Feb 8 2023 9:15AM,256897,10,"Methapharm, Inc.",,,17
9,Feb 8 2023 9:09AM,256892,18,Innova Softgel LLC,,1,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 8 2023 9:54AM,256904,15,"Alliance Pharma, Inc.",34,,
1,Feb 8 2023 9:52AM,256903,19,"Snap Medical Industries, LLC",1,,
2,Feb 8 2023 9:51AM,256902,10,"Snap Medical Industries, LLC",2,,
3,Feb 8 2023 9:49AM,256901,19,"Methapharm, Inc.",2,,
4,Feb 8 2023 9:45AM,256827,15,"Virtus Pharmaceuticals, LLC",16,,
5,Feb 8 2023 9:33AM,256900,15,"Brookfield Pharmaceuticals, LLC",16,,
6,Feb 8 2023 9:28AM,256899,10,"Citieffe, Inc.",1,,
7,Feb 8 2023 9:25AM,256898,22,"NBTY Global, Inc.",1,,
8,Feb 8 2023 9:15AM,256897,10,"Methapharm, Inc.",17,,
9,Feb 8 2023 9:09AM,256892,18,Innova Softgel LLC,,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 8 2023 9:54AM,256904,15,"Alliance Pharma, Inc.",34,,
1,Feb 8 2023 9:52AM,256903,19,"Snap Medical Industries, LLC",1,,
2,Feb 8 2023 9:51AM,256902,10,"Snap Medical Industries, LLC",2,,
3,Feb 8 2023 9:49AM,256901,19,"Methapharm, Inc.",2,,
4,Feb 8 2023 9:45AM,256827,15,"Virtus Pharmaceuticals, LLC",16,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 8 2023 9:54AM,256904,15,"Alliance Pharma, Inc.",34.0,NaN,NaN
1,Feb 8 2023 9:52AM,256903,19,"Snap Medical Industries, LLC",1.0,NaN,NaN
2,Feb 8 2023 9:51AM,256902,10,"Snap Medical Industries, LLC",2.0,NaN,NaN
3,Feb 8 2023 9:49AM,256901,19,"Methapharm, Inc.",2.0,NaN,NaN
4,Feb 8 2023 9:45AM,256827,15,"Virtus Pharmaceuticals, LLC",16.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 8 2023 9:54AM,256904,15,"Alliance Pharma, Inc.",34.0,0.0,0.0
1,Feb 8 2023 9:52AM,256903,19,"Snap Medical Industries, LLC",1.0,0.0,0.0
2,Feb 8 2023 9:51AM,256902,10,"Snap Medical Industries, LLC",2.0,0.0,0.0
3,Feb 8 2023 9:49AM,256901,19,"Methapharm, Inc.",2.0,0.0,0.0
4,Feb 8 2023 9:45AM,256827,15,"Virtus Pharmaceuticals, LLC",16.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4623586,171.0,28.0,1.0
15,1027414,93.0,0.0,25.0
16,770643,3.0,0.0,0.0
18,256892,0.0,1.0,0.0
19,2054937,31.0,19.0,0.0
21,770555,2.0,1.0,0.0
22,256898,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4623586,171.0,28.0,1.0
1,15,1027414,93.0,0.0,25.0
2,16,770643,3.0,0.0,0.0
3,18,256892,0.0,1.0,0.0
4,19,2054937,31.0,19.0,0.0
5,21,770555,2.0,1.0,0.0
6,22,256898,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,171.0,28.0,1.0
1,15,93.0,0.0,25.0
2,16,3.0,0.0,0.0
3,18,0.0,1.0,0.0
4,19,31.0,19.0,0.0
5,21,2.0,1.0,0.0
6,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,171.0
1,15,Released,93.0
2,16,Released,3.0
3,18,Released,0.0
4,19,Released,31.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,21,22
Status,,,,,,,
Completed,1.0,25.0,0.0,0.0,0.0,0.0,0.0
Executing,28.0,0.0,0.0,1.0,19.0,1.0,0.0
Released,171.0,93.0,3.0,0.0,31.0,2.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,21,22
0,Completed,1.0,25.0,0.0,0.0,0.0,0.0,0.0
1,Executing,28.0,0.0,0.0,1.0,19.0,1.0,0.0
2,Released,171.0,93.0,3.0,0.0,31.0,2.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,21,22
0,Completed,1.0,25.0,0.0,0.0,0.0,0.0,0.0
1,Executing,28.0,0.0,0.0,1.0,19.0,1.0,0.0
2,Released,171.0,93.0,3.0,0.0,31.0,2.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()